In [1]:
import sys
sys.path.append('../')


import os
import pickle
import numpy as np

from onsets import read_audio, onset_detection_fn, reduce_frequency_bands_in_spectrogram,get_onset_detect,get_grid_timestamps,map_onsets_to_grid

In [2]:
source_path = "../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs"
print(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"))
train_file = open(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_sequence_data.obj"),'rb')
train_set = pickle.load(train_file)
dataset_size = len(train_set)
# ix =  int(np.random.random_sample()*dataset_size)
example = train_set[52] # 4-4 120 bpm 2 bars
example.to_html_plot(show_figure=True)

../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs/GrooveMIDI_processed_train/hvo_data.obj


Figure(id='1002', ...)

In [3]:
sr=44100
audio_file_path = "./misc/temp.wav"

# IN ORDER TO CALC HOP SIZE
FRAME_INTERVAL = 0.01  # s

x, sr = read_audio(audio_file_path, mono=True, sr=sr)
x /= np.max(np.abs(x))

n_fft= 1024
f_win_size = 1024
f_hop_size = int(round(FRAME_INTERVAL * sr))
f_bins_per_octave =16
f_octaves = 9
f_fmin = 40
mean_filter_size = 22



od_fun, logf_stft, f_cq = onset_detection_fn(x,
                                             f_win_size,
                                             f_hop_size,
                                             f_bins_per_octave,
                                             f_octaves,
                                             f_fmin,
                                             sr,
                                             mean_filter_size)

mb_onset_strength = od_fun

c_freq = [55, 90, 138, 175, 350, 6000,8500,12500]                      # Perfe center frequencies
mb_onset_strength = reduce_frequency_bands_in_spectrogram(c_freq, f_cq, mb_onset_strength)
mb_onset_detect = get_onset_detect(mb_onset_strength)
grid = get_grid_timestamps(n_bars = 2, time_signature_numerator = 4, 
                                      time_signature_denominator = 4, beat_division_factors = [4],qpm = 120)

onsets_grid, strength_grid = map_onsets_to_grid(grid, mb_onset_strength, mb_onset_detect, n_fft=n_fft, hop_length = f_hop_size,sr=sr)

../hvo_sequence/hvo_seq.py:92: UserWarning: drum_mapping is not specified
  warnings.warn("drum_mapping is not specified")


In [4]:
print(np.round(onsets_grid,2))

[[ 0.33  0.33  0.25  0.25  0.41  0.33  0.17  0.17]
 [ 0.37  0.37 -0.11  0.    0.05  0.37  0.21  0.37]
 [ 0.    0.    0.    0.    0.    0.    0.33  0.41]
 [ 0.    0.    0.    0.   -0.43  0.   -0.27  0.29]
 [ 0.49  0.49  0.09  0.33  0.01  0.25  0.41  0.25]
 [ 0.29  0.29 -0.19  0.    0.   -0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.01  0.17]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.49  0.25  0.01  0.01  0.01  0.01  0.01  0.01]
 [ 0.05  0.13 -0.35  0.    0.    0.05  0.    0.  ]
 [ 0.    0.    0.   -0.07  0.    0.01  0.25  0.17]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.41  0.41  0.49  0.49  0.17  0.17  0.49  0.33]
 [ 0.21  0.29 -0.19  0.    0.    0.   -0.35 -0.43]
 [ 0.    0.    0.    0.    0.    0.25  0.25  0.41]
 [ 0.    0.21  0.21  0.21  0.21  0.    0.    0.  ]
 [ 0.41  0.41  0.17  0.17  0.17  0.17  0.17  0.17]
 [ 0.21  0.29 -0.27  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.09  0.09  0.25]
 [ 0.    0.    0.    0.    0.  

In [5]:
print(np.round(strength_grid, 2))

[[0.98 0.96 0.53 0.49 0.45 0.6  0.88 0.86]
 [0.4  0.25 0.34 0.   0.27 0.7  0.82 0.22]
 [0.   0.   0.   0.   0.   0.   0.42 0.78]
 [0.   0.   0.   0.   0.18 0.   0.47 0.19]
 [0.92 0.82 0.95 0.8  0.51 0.91 0.93 0.19]
 [0.37 0.36 0.27 0.   0.   0.57 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.64 0.65]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.65 0.76 0.8  0.63 0.35 0.42 0.16 0.3 ]
 [0.24 0.19 0.35 0.   0.   0.12 0.   0.  ]
 [0.   0.   0.   0.11 0.   0.28 0.38 0.93]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.87 0.78 0.64 0.68 0.87 0.66 0.78 0.28]
 [0.35 0.09 0.31 0.   0.   0.   0.45 0.2 ]
 [0.   0.   0.   0.   0.   0.12 0.68 0.66]
 [0.   0.12 0.18 0.22 0.21 0.   0.   0.  ]
 [0.84 0.77 0.8  0.63 0.36 0.41 0.17 0.31]
 [0.28 0.22 0.26 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.14 0.52 0.43]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.87 0.8  0.71 0.75 0.85 1.   0.96 0.42]
 [0.37 0.16 0.33 0.   0.   0.71 0.59 0.  ]
 [0.   0.   0.   0.   0.   0.1  0.75 0.78]
 [0.   0.  